# PWV effects on SN Magnitude

This notebook demonstrates the effects of PWV absorption on the apparent magnitude of SNe.


In [ ]:
import sys
sys.path.insert(0, '../')

import numpy as np
import sncosmo
from astropy.cosmology import FlatLambdaCDM
from matplotlib import pyplot as plt
from sn_analysis import modeling, sn_magnitudes, reference, plotting
from utils import register_decam_filters

register_decam_filters(force=True)


In [ ]:
source = 'salt2-extended'
pwv_vals = np.arange(0, 10)
z_vals = np.arange(.01, 1.1, .05)
bands = 'decam_r', 'decam_i', 'decam_z', 'decam_y'


## Spectral Template

The effect of PWV absorption is fundementally dependent on the interplay between the SED of the effected object and the PWV transmission function. For this work we consider an extend version of the Salt2 spectral template. We visualize the template as a reference.


In [ ]:
_ = plotting.plot_salt2_extended_template(np.arange(4000, 10000), [0, .5, 1], pwv=4)


## Apparent magnitude

We start by considering the direct impact of PWV absorbtion on simulated SN Ia magnitudes. Apparent magnitudes are simulated for multiple bands, redshifts, and PWV concentrations. 


In [ ]:
tabulated_mag = sn_magnitudes.tabulate_mag(source, pwv_vals, z_vals, bands)


In [ ]:
_ = plotting.plot_magnitude(tabulated_mag, pwv_vals, z_vals)

---

**Sanity Check:** As the spectral template is redshifted, the brighter portion of the spectra begins to enter the redder bands. We expect this to cause a temporary decrease in the apparent magnitude until the redshift becomes large enough that growing distance results in a monotonically increasing magnitude (This effect becomes more obvious when looking at the spectra template above).

---

To better understand the impact of PWV, we estimate the change in apparent magnitude due to PWV. Although we could measure this change relative to PWV=0, a more physically motivated approach is to use a fiducial atmosphere with a non-zero PWV component:

$$\Delta m = m(\text{PWV}, z) - m(\text{PWV}_f, z)$$

We also determine the slope in the apparent magnitude as an estimate for how sensitive our simulated observations are to PWV fluxtuations:

$$\frac{\Delta m}{\Delta \text{PWV}}(z) = \frac{m(\text{PWV}_2, z) - m(\text{PWV}_1, z)}{\text{PWV}_2 - \text{PWV}_1}$$

Here PWV$_1$ and PWV$_2$ are chosen to be equidistant to PWV$_f$. We take note of the chosen PWV reference values in the following cell:


In [ ]:
reference_pwv_config = reference.get_config_pwv_vals()
print(reference_pwv_config)


We tabulate values for $\Delta m$ and $\frac{\Delta m}{\Delta \text{PWV}}(z)$. 

In [ ]:
fiducial_mag = sn_magnitudes.tabulate_fiducial_mag(
        source, z_vals, bands, reference_pwv_config)

tabulated_delta_mag, tabulated_slope = sn_magnitudes.calc_delta_mag(
    tabulated_mag, fiducial_mag, reference_pwv_config)


In [ ]:
_ = plotting.plot_pwv_mag_effects(
    pwv_vals, 
    z_vals, 
    tabulated_delta_mag, 
    tabulated_slope, 
    bands)


---

**Sanity Check:** We expect to see the following trends in the above plot:
- The bluer bands should have minimal PWV impact. 
- The size of $\Delta$m should be largest for the redder bands. 
- The slope in $\Delta$m should be largest in the redder bands, and almost zero in the blewer bands.
- $\Delta$m should be zero for the fiducial PWV value.

---

## $\Delta m$ With Fitting

Instead of tabulating the simulating magnitude, we can instead consider the change in magnitude by simulating light-curves with PWV effects and then fitting a model without a PWV component.

In [ ]:
# Realize simulations of light-curves
observations = modeling.create_observations_table()
light_curves = modeling.iter_lcs(observations, source, pwv_vals, z_vals)

# Fit light curves
vparams = ['x0', 'x1', 'c']
fitted_mag, fitted_params = sn_magnitudes.fit_mag(
        source, light_curves, vparams, pwv_vals, z_vals, bands)

# Get fiducial mag (uncalibrated)
fitted_fiducial_mag, fitted_fiducial_params = sn_magnitudes.fit_fiducial_mag(
        source, observations, vparams, z_vals, bands, reference_pwv_config)


Here we visualize the fitted, uncalibrated (i.e. without any stretch / color corrections) magnitudes.

In [ ]:
fig, axes = plotting.plot_magnitude(fitted_mag, pwv_vals, z_vals)
fig.suptitle('Fitted Magnitude', y=1.05)


---

**Sanity Check:** The fitted light-curves are constructed to have a dense, uniform sampling. As a result, the fitted magnitude should thus look extremly similar to the tabulated magnitudes from earlier in the notebook.

---

We expect to see almost no difference between the tabulated, and fitted magnitudes in each band. However, we do expect to variation in the fitted stretch and color.

In [ ]:
# Parse the fitted parameters for easier plotting
model = sncosmo.Model('salt2-extended')
params_dict = {
    param: fitted_params[bands[0]][..., i] for 
    i, param in enumerate(model.param_names)
}

fig, axes = plt.subplots(2, 3, figsize=(12, 8))
for axis, (param, param_vals) in zip(axes.flatten(), params_dict.items()):
    plotting.multi_line_plot(z_vals, param_vals, pwv_vals, axis)
    axis.set_xlabel('Redshift')
    axis.set_ylabel(param)
    
correction_factor = sn_magnitudes.alpha * params_dict['x1'] - sn_magnitudes.beta * params_dict['c']
plotting.multi_line_plot(z_vals, correction_factor, pwv_vals, axes[-1][-1])

label = f'{sn_magnitudes.alpha} * $x_1$ - {sn_magnitudes.beta} * $c$'
axes[-1][-1].set_ylabel(label)
    
plt.tight_layout()


Like before we look at the impact of PWV on the fitted magnitude, but we also look at the cirrected magnitude.

In [ ]:
corrected_delta_mag, corrected_slope = sn_magnitudes.calc_delta_mag(
    fitted_mag, fitted_fiducial_mag, reference_pwv_config)


In [ ]:
fig, axes = plotting.plot_pwv_mag_effects(
    pwv_vals, 
    z_vals, 
    corrected_delta_mag, 
    corrected_slope, 
    bands)


Next we add in the alpha and beta parameters from the fit.

In [ ]:
# Determine calibrated magnitude from fits
calibrated_mag = {}
calibrated_fiducial_mag = {}
for band in bands:
    calibrated_mag[band] = sn_magnitudes.calibrate_mag(
        source, fitted_mag[band], fitted_params[band])

    # Get fiducial mag (calibrated)
    calibrated_fiducial_mag[band] = sn_magnitudes.calibrate_mag(
        source, fitted_fiducial_mag[band], fitted_fiducial_params[band])

calibrated_delta_mag, calibrated_slope = sn_magnitudes.calc_delta_mag(
        calibrated_mag, calibrated_fiducial_mag, reference_pwv_config)


In [ ]:
_ = plotting.plot_pwv_mag_effects(
    pwv_vals, 
    z_vals, 
    calibrated_delta_mag, 
    calibrated_slope, 
    bands)


# Relative to reference Star

In practice flux values are calibrated relative to a reference star. To understand how PWV effects SNe fluxes under these conditions, we normalize the SNe and Reference star flux to their respective fluxes through the fiducial atmosphere and take the difference (Normalized SNe - normalized reference star).

Note that the value of $\Delta m$ is we calculated before is equivilent to the apparent SN magnitude normalized to the flux through the fiducial atmosphere.

$$\Delta m = m(\text{PWV}, z) - m(\text{PWV}_f, z) = -2.5 log\left(\frac{f(\text{PWV}, z)}{f(\text{PWV}_f, z)}\right)$$

In [ ]:
def calibrate_to_reference(delta_mag, slope, pwv_vals, reference_type='G2'):
    
    assert delta_mag.keys() == slope.keys()
    
    delta_mag_ref = {}
    slope_ref = {}
    for band in delta_mag:
        
        delta_mag_ref[band] = reference.subtract_ref_star(
            band, delta_mag[band], pwv_vals, reference_type)
        
        slope_ref[band] = reference.subtract_ref_star_slope(
            band, slope[band], reference_pwv_config, reference_type)
        
    return delta_mag_ref, slope_ref
        

In [ ]:
# The y band values are not available for the reference star
# we drop them here

_tabulated_delta_mag = tabulated_delta_mag.copy()
_tabulated_delta_mag.pop('decam_y')

_tabulated_slope = tabulated_slope.copy()
_tabulated_slope.pop('decam_y')

tabulated_delta_mag_g2 = reference.subtract_ref_star_dict(_tabulated_delta_mag, pwv_vals)
tabulated_slope_g2 = reference.subtract_ref_star_slope(_tabulated_slope, reference_pwv_config)


In [ ]:
_ = plotting.plot_pwv_mag_effects(
    pwv_vals, 
    z_vals, 
    tabulated_delta_mag_g2, 
    tabulated_slope_g2, 
    bands[:-1])


In [ ]:
_corrected_delta_mag = corrected_delta_mag.copy()
_corrected_delta_mag.pop('decam_y')

_corrected_slope = corrected_slope.copy()
_corrected_slope.pop('decam_y')

corrected_delta_mag_g2 = reference.subtract_ref_star_dict(_corrected_delta_mag, pwv_vals)
corrected_slope_g2 = reference.subtract_ref_star_slope(_corrected_slope, reference_pwv_config)


In [ ]:
_ = plotting.plot_pwv_mag_effects(
    pwv_vals, 
    z_vals, 
    corrected_delta_mag_g2,
    corrected_slope_g2,
    bands)


Note that in the above plot we have subtracted off the reference star **after** fitting the light-curve. In principle we should have subtracted of the reference beforehand, as follows:


In [ ]:
def iter_light_curve_iter_with_ref(source, pwv_arr, z_arr, reference_type='G2'):
    
    bands = ('decam_r', 'decam_i', 'decam_z')
    
    # Unfortunatly we cant specify a redshift dependent zero point
    # in the observations table so we build an iterater over some lightcurves
    # and recalibrate to a new zeropoint retroactively
    initial_zeropoint = 25
    observations = modeling.create_observations_table(bands=bands, zp=initial_zeropoint)
    light_curves = modeling.iter_lcs(observations, source, pwv_arr, z_arr)
    
    # Determine what the reference mag should be
    print('Fitting for fiducial mag', flush=True)
    vparams = ['x0', 'x1', 'c']
    fiducial_mag_dict, _ = sn_magnitudes.fit_fiducial_mag(
        source, observations, vparams, z_arr, bands, reference_pwv_config)
    
    for lc in light_curves:
        z = lc.meta['z']
        for band, fiducial_magnitudes in fiducial_mag_dict.items():
            _, fiducial_mag, _  = fiducial_magnitudes
            zp_for_z = fiducial_mag[np.where(z_arr == z)[0]]
            
            indices = lc['band'] == band
            lc['flux'][indices] *= 10 ** ((zp_for_z - initial_zeropoint) / 2.5)
            lc['fluxerr'][indices] *= 10 ** ((zp_for_z - initial_zeropoint) / 2.5)
            lc['zp'][indices] = zp_for_z
            
        yield lc
            
    

In [ ]:
light_curves_with_ref = iter_light_curve_iter_with_ref(source, pwv_vals, z_vals)
ref_first_fitted_mag, ref_first_fitted_params = sn_magnitudes.fit_mag(
        source, light_curves_with_ref, vparams, pwv_vals, z_vals, bands)


In [ ]:
# Determine calibrated magnitude from fits
ref_first_calibrated_mag = {}
ref_first_calibrated_fiducial_mag = {}
for band in bands:
    ref_first_calibrated_mag[band] = sn_magnitudes.calibrate_mag(
        source, ref_first_fitted_mag[band], ref_first_fitted_params[band])


## Delta $\mu$


In [ ]:
# We use cosmological parameters from Betoule 14
betoule_cosmo = FlatLambdaCDM(H0=sn_magnitudes.H0, Om0=sn_magnitudes.omega_m)
betoule_cosmo


In [ ]:
# Nore for the following line that params are band independent
fitted_mu = sn_magnitudes.calc_mu_for_params(source, fitted_params['decam_r'])
corrected_fitted_mu = fitted_mu + correction_factor


In [ ]:
def plot_mu(mu, cosmo, pwv_arr, z_arr, xval='redshift'):
    
    cosmo_mu = cosmo.distmod(z_arr).value
    delta_mu = mu - cosmo_mu
    corrected_delta_mu = delta_mu - cosmo_mu

    fig, axes = plt.subplots(1, 3, figsize=(9, 3))
    mu_ax, delta_mu_ax, relative_mu_ax = axes

    plotting.multi_line_plot(z_arr, mu, pwv_arr, mu_ax)
    mu_ax.plot(z_arr, cosmo_mu, linestyle='--', color='k', label='Betoule 14')
    mu_ax.legend(framealpha=1)

    plotting.multi_line_plot(z_arr, delta_mu, pwv_arr, delta_mu_ax)
    delta_mu_ax.axhline(0, linestyle='--', color='k', label='Betoule 14')
    delta_mu_ax.legend(framealpha=1)

    plotting.multi_line_plot(z_arr, mu - mu[4], pwv_arr, relative_mu_ax, label='{:g} mm')
    relative_mu_ax.axhline(0, linestyle='--', color='k', label=f'PWV={pwv_arr[4]}')
    relative_mu_ax.legend(framealpha=1, bbox_to_anchor=(1, 1.1))

    mu_ax.set_ylabel(r'$\mu$', fontsize=12)
    delta_mu_ax.set_ylabel(r'$\mu - \mu_{cosmo}$', fontsize=12)
    relative_mu_ax.set_ylabel(r'$\mu - \mu_{pwv_f}$', fontsize=12)
    for ax in axes:
        ax.set_xlabel('Redshift', fontsize=12)

    plt.tight_layout()


In [ ]:
plot_mu(fitted_mu, betoule_cosmo, pwv_vals, z_vals)


In [ ]:
plot_mu(corrected_fitted_mu, betoule_cosmo, pwv_vals, z_vals)
